In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com'

# Load data

In [ ]:
imperv = ee.Image("Tsinghua/FROM-GLC/GAIA/v10").select('change_year_index');
impervScale = imperv.projection().nominalScale()

# Map.addLayer(imperv, {}, "Impervious in 2018");

impervmask = imperv.updateMask(imperv.gte(0))
# Map.addLayer(impervmask,{},"impervious areas")

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir+'/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract impervious by city

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir+'/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image of albedo as GeoTIFF to Google Drive 
    geemap.ee_export_image_to_drive(
        impervmask.toByte(), # use toByte() to reduce file size
        description = boundary_id + '-impervious-areas-through-2018',
        folder='data', 
        scale=impervScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id + '-impervious-areas-through-2018.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/impervious/tsinghua/v_0/' + boundary_id + '-impervious-areas-through-2018.tif',
                               ExtraArgs={'ACL':'public-read'})

# Visualize data

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=8)

In [ ]:
Map.addLayer(greenmask),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Vegetation cover',True,1)